In [31]:
import pandas as pd
import os
os.chdir("C:/Users/pavan/Desktop/sony sentiment analysis")
train = pd.read_csv("Data/train.csv")

In [32]:
train.head(5)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [33]:
# statistics
train.sentiment.value_counts()

neutral     11118
positive     8582
negative     7781
Name: sentiment, dtype: int64

In [34]:
# clean text, remove url link, stop words, stem words¶
import re
import string
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [35]:
train["text"] = train["text"].apply(lambda x: clean_text(x))
train['selected_text'] = train['selected_text'].apply(lambda x:clean_text(x))

In [36]:
# Train Spacy Name Entity Recognition (NER)

def get_train_datas(data):
    train_datas = []
    texts = data.text
    selected_texts = data.selected_text
    for selected_text, text in zip(selected_texts, texts):
        start = text.find(selected_text)
        end = start + len(selected_text)
        train_datas.append((text, {"entities":[(start, end, "selected_text")]}))
    return train_datas

In [37]:
def load_model(pre_model = None, label = None):
    if pre_model is not None:
        nlp = spacy.load(pre_model)
        print("Loaded model '%s'" % pre_model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

        if "ner" not in nlp.pipe_names:
            ner = nlp.create_pipe("ner")
            nlp.add_pipe(ner)
        else:
            ner = nlp.get_pipe("ner")

        if label is not None:
            ner.add_label(label)

    return nlp

In [38]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
Positive_sent = train[train['sentiment']=='positive']
Negative_sent = train[train['sentiment']=='negative']
Neutral_sent = train[train['sentiment']=='neutral']

Positive_sent = Positive_sent.reindex(axis=1)
Positive_sent["text"] = Positive_sent["text"].apply(lambda x: x.strip())
Negative_sent = Negative_sent.reindex(axis=1)
Negative_sent["text"] = Negative_sent["text"].apply(lambda x: x.strip())

Positive_sent["text"] = Positive_sent["text"].apply(lambda text: ' '.join(text_to_word_sequence(
    text, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' '
)))
Negative_sent["text"] = Negative_sent["text"].apply(lambda text: ' '.join(text_to_word_sequence(
    text, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' '
)))

positive_tweet = get_train_datas(Positive_sent)
negative_tweet = get_train_datas(Negative_sent)

positive_tweet_train = []
for i in range(len(positive_tweet)):
    if positive_tweet[i][0] != "":
        positive_tweet_train.append(positive_tweet[i])
        
negative_tweet_train = []
for i in range(len(negative_tweet)):
    if negative_tweet[i][0] != "":
        negative_tweet_train.append(positive_tweet[i])

In [39]:
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
   
ps = PorterStemmer() 

def stem_word(words):
    text = []
    for word in word_tokenize(words):
        text.append(ps.stem(word))
    return " ".join(text)
train["text"] = train["text"].apply(lambda x: stem_word(x))
train['selected_text'] = train['selected_text'].apply(lambda x:stem_word(x))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pavan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
from collections import Counter
cnt = Counter()
for line in range(len(train)):
    for word in word_tokenize(train.iloc[line, 1]):
        cnt[word] += 1

In [41]:
cnt.most_common(20)

[('i', 13169),
 ('to', 10013),
 ('the', 8981),
 ('a', 6710),
 ('it', 6064),
 ('my', 5510),
 ('and', 5067),
 ('you', 4812),
 ('is', 3973),
 ('in', 3787),
 ('for', 3650),
 ('of', 3144),
 ('that', 3119),
 ('im', 3023),
 ('have', 2898),
 ('on', 2850),
 ('me', 2820),
 ('so', 2575),
 ('day', 2368),
 ('go', 2363)]

In [42]:
from collections import Counter
cnt = Counter()
for line in range(len(Positive_sent)):
    for word in word_tokenize(Positive_sent.iloc[line, 2]):
        cnt[word] += 1
cnt.most_common(10)

[('i', 1040),
 ('good', 826),
 ('happy', 730),
 ('love', 697),
 ('you', 623),
 ('to', 608),
 ('a', 589),
 ('the', 571),
 ('day', 456),
 ('thanks', 439)]

In [43]:
from collections import Counter
cnt = Counter()
for line in range(len(Negative_sent)):
    for word in word_tokenize(Negative_sent.iloc[line, 2]):
        cnt[word] += 1
cnt.most_common(10)

[('i', 1333),
 ('to', 594),
 ('the', 547),
 ('my', 524),
 ('a', 472),
 ('im', 452),
 ('not', 421),
 ('is', 373),
 ('so', 360),
 ('miss', 358)]

In [44]:
from collections import Counter
cnt = Counter()
for line in range(len(Neutral_sent)):
    for word in word_tokenize(Neutral_sent.iloc[line, 2]):
        cnt[word] += 1
cnt.most_common(10)

[('i', 4827),
 ('to', 4103),
 ('the', 3472),
 ('a', 2477),
 ('my', 1971),
 ('and', 1800),
 ('you', 1760),
 ('in', 1574),
 ('it', 1476),
 ('is', 1470)]

In [45]:
import os
def test_model(ner_model, text):
    doc = ner_model(text)
    ent_array = []
    for ent in doc.ents:
        start = text.find(ent.text)
        end = start + len(ent.text)
        new_int = [start, end, ent.label_]
        
        if new_int not in ent_array:
            ent_array.append([start, end, ent.label_])
        
    return text[ent_array[0][0]:ent_array[0][1]] if len(ent_array) > 0 else text


def save_model(ner_model, output_dir = None, new_model_name = None):
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        ner_model.meta["name"] = new_model_name
        ner_model.to_disk(output_dir)
        print("Saved model to", output_dir)

In [46]:
def get_model(sentiment, train_datas, more_iters = 30):
    if sentiment == 'positive':
        positive_model_path = "model"
        positive_datas = train_datas
        if not os.path.exists(positive_model_path):
            nlp = load_model(label = 'selected_text')
            ner_model_positive = train_model(None, nlp, positive_datas, n_iter=50)
            save_model(ner_model_positive, output_dir = "model", new_model_name = "posi_model")
        else:
            ner_model_positive = load_model(positive_model_path)
#             ner_model_positive = spacy.load("/content/sample_data/positive_ner")
            if more_iters > 0:
                ner_model_positive = train_model(positive_model_path, ner_model_positive, positive_datas, more_iters)
                save_model(ner_model_positive, output_dir = "model", new_model_name = "posi_model")
        return ner_model_positive
    else:
        negative_model_path = "model"
        negative_datas = train_datas
        if not os.path.exists(negative_model_path):
            nlp = load_model(label = 'selected_text')
            ner_model_negative = train_model(None, nlp, negative_datas, n_iter=50)
            save_model(ner_model_negative, output_dir = "model", new_model_name = "nega_model")
        else:
            ner_model_negative = load_model(negative_model_path)
#             ner_model_negative = spacy.load("/content/sample_data/negative_ner")
            if more_iters > 0:
                ner_model_negative = train_model(negative_model_path, ner_model_negative, negative_datas, more_iters)
                save_model(ner_model_negative, output_dir = "model", new_model_name = "nega_model")
        return ner_model_negative

In [47]:
def train_model(model, nlp, train_datas, n_iter = 30):
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

    with nlp.disable_pipes(*other_pipes):
        sizes = compounding(1.0, 64.0, 1.001)
        print(train_datas[0])
        for itn in range(n_iter):
            random.shuffle(train_datas)
            batches = minibatch(train_datas, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print(itn, "Losses", losses)

    return nlp

In [ ]:
from __future__ import unicode_literals, print_function
#!pip install spacy
import spacy
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
ner_model_positive = get_model('positive', positive_tweet_train, more_iters=100)

Loaded model 'model'
('feedings for the baby are fun when he is all smiles and coos', {'entities': [(26, 29, 'selected_text')]})
0 Losses {'ner': 14105.741679266639}
1 Losses {'ner': 11791.706565968343}
2 Losses {'ner': 10790.258924379945}
3 Losses {'ner': 9667.56991237402}
4 Losses {'ner': 9473.406091213226}
5 Losses {'ner': 9247.695692300797}
6 Losses {'ner': 8788.337981387973}
7 Losses {'ner': 8568.32497882843}
8 Losses {'ner': 8242.65180554646}
9 Losses {'ner': 8196.93265517801}
10 Losses {'ner': 8157.148842210823}
11 Losses {'ner': 8048.615706324577}
12 Losses {'ner': 8059.8176928162575}
13 Losses {'ner': 7805.235530148417}
14 Losses {'ner': 7928.651595549443}
15 Losses {'ner': 7845.625191949455}
16 Losses {'ner': 7669.27836596081}
17 Losses {'ner': 7430.36457942985}
18 Losses {'ner': 7532.837741696276}
19 Losses {'ner': 7443.110233053565}
20 Losses {'ner': 7475.347391770221}
21 Losses {'ner': 7233.652051564306}
22 Losses {'ner': 7361.672007016838}
23 Losses {'ner': 7304.014220950

In [ ]:
negative_tweet_tr = []
for i in negative_tweet_train:
    if i[0] != '':
        negative_tweet_tr.append(i)

In [ ]:
ner_model_negative = get_model('negative', negative_tweet_tr, more_iters=50)

Loaded model '/content/sample_data/positive_ner'
('we should have a twitter reunion it would be awesom to meet you all lol iwond howd iget that to pull off', {'entities': [(45, 51, 'selected_text')]})
0 Losses {'ner': 16260.894958229415}
1 Losses {'ner': 13873.118352340167}
2 Losses {'ner': 12229.384261280298}
3 Losses {'ner': 11585.15696176514}
4 Losses {'ner': 10908.007576373406}
5 Losses {'ner': 10961.337362468243}
6 Losses {'ner': 10709.637624919415}
7 Losses {'ner': 10199.906262397766}
8 Losses {'ner': 9839.907517433167}
9 Losses {'ner': 9844.898794412613}
10 Losses {'ner': 9533.167648792267}
11 Losses {'ner': 9458.761201381683}
12 Losses {'ner': 9280.555729866028}
13 Losses {'ner': 9119.501657247543}
14 Losses {'ner': 9258.757742643356}
15 Losses {'ner': 8978.34073805809}
16 Losses {'ner': 8784.205489754677}
17 Losses {'ner': 9090.147297143936}
18 Losses {'ner': 8755.651048183441}
19 Losses {'ner': 8565.589799523354}
20 Losses {'ner': 8791.473707556725}
21 Losses {'ner': 8835.208

In [ ]:
#test
test_dataset = pd.read_csv("test.csv")

In [ ]:
test_dataset["text"] = test_dataset["text"].apply(lambda x: x.strip())
test_dataset['n_text_words'] = test_dataset['text'].apply(lambda text: len(str(text).split()))

In [ ]:
pre_list = []
for i in range(test_dataset.shape[0]):
    t_data = test_dataset.iloc[i]
    if t_data.sentiment == 'neutral' or t_data.n_text_words <= 3:
        pre_list.append(t_data.text)
    elif t_data.sentiment == 'positive':
        pre_list.append(test_model(ner_model_positive, t_data.text))
    else:
        pre_list.append(test_model(ner_model_negative, t_data.text))

In [ ]:
submission = pd.read_csv("/content/sample_submission.csv")

In [ ]:
submission['selected_text'] = pre_list

In [ ]:
submission.head(10)

,textID,selected_text
0,f87dea47db,Last session of the day http://twitpic.com/67ezh
1,96d74cb729,Good
2,eee518ae67,shame!
3,01082688c6,happy bday!
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!
5,726e501993,that`s great!! weee!! visitors!
6,261932614e,I THINK EVERYONE HATES ME ON HERE lol
7,afa11da83f,"soooooo wish i could, but im in school and mys..."
8,e64208b4ef,and within a short time of the last clue all o...
9,37bcad24ca,What did you get? My day is alright.. haven`t...


In [ ]:
submission.to_csv("./submision_final.csv")